In [2]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import *
import numpy as np
import sys, os
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from workspace._global.config import *
from workspace._global.funcs import *
from workspace._global.bbx import *
from workspace._global.recognition import *
sys.path.extend([RESEARCH])
from object_detection.utils import config_util
import matplotlib.pyplot as plt
from object_detection.builders import model_builder
from IPython import get_ipython
import cv2 
import tensorflowjs as tfjs
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
ipython = get_ipython()

In [3]:
def find_and_extract_faces(input_path, output_path, model):
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    imgs = [(f ,cv2.imread(os.path.join(input_path, f))) for f in os.listdir(input_path)]
    for i in range(len(imgs)):
        filename, img = imgs[i]
        find_and_extract_face(model, img, os.path.join(output_path, filename))

In [4]:
INIT_IMGS_1 = os.path.join(FACES_RECOG_IMG, "INIT_FACES_1")
INIT_IMGS_2 = os.path.join(FACES_RECOG_IMG, "INIT_FACES_2")
INIT_IMGS_CROPPED_1 = os.path.join(FACES_RECOG_IMG, "INIT_FACES_CROPPED_1")
INIT_IMGS_CROPPED_2 = os.path.join(FACES_RECOG_IMG, "INIT_FACES_CROPPED_2")
INIT_IMGS_TFRECORD_1 = os.path.join(FACE_RECOG_DATA, "INIT_FACES_1.tfrecord")
INIT_IMGS_TFRECORD_2 = os.path.join(FACE_RECOG_DATA, "INIT_FACES_2.tfrecord")
INIT_IMGS_1_LABEL_NAME = "Init_Img_1"
INIT_IMGS_2_LABEL_NAME = "Init_Img_2"
INIT_IMGS_1_LABEL_NUM = 1
INIT_IMGS_2_LABEL_NUM = 2
LAST_DENSE_BEFORE_OUTPUT_NEURONAS = 512

config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
BBX_LOCALIZER = model_builder.build(model_config=config['model'], is_training=False)
ckpt = tf.compat.v2.train.Checkpoint(model=BBX_LOCALIZER) 
ckpt.restore(os.path.join(MODEL, 'ckpt-12')).expect_partial()


In [5]:
find_and_extract_faces(INIT_IMGS_1, INIT_IMGS_CROPPED_1, BBX_LOCALIZER)
find_and_extract_faces(INIT_IMGS_2, INIT_IMGS_CROPPED_2, BBX_LOCALIZER)

In [6]:
parse_images(INIT_IMGS_TFRECORD_1, INIT_IMGS_CROPPED_1, INIT_IMGS_1_LABEL_NAME, INIT_IMGS_1_LABEL_NUM, 3)
parse_images(INIT_IMGS_TFRECORD_2, INIT_IMGS_CROPPED_2, INIT_IMGS_2_LABEL_NAME, INIT_IMGS_2_LABEL_NUM, 3)

In [7]:
INIT_DATASET = tf.data.TFRecordDataset([INIT_IMGS_TFRECORD_1,INIT_IMGS_TFRECORD_2])
INIT_DATASET = INIT_DATASET.map(parse_fn)
INIT_DATASET = INIT_DATASET.shuffle(330)
INIT_DATASET = INIT_DATASET.batch(FACE_RECOG_BATCH_SIZE)

c:\xampp\htdocs\projects\Face_Recognition_Ullsperger\LogicAIFaceRecProject\tf\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:4526: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [8]:
for i in tf.compat.v1.python_io.tf_record_iterator(INIT_IMGS_TFRECORD_1):
        img, label = parse_fn(i)
        label = label.numpy()
        num =  img.numpy()
        plt.imsave(os.path.join(INSTALLATION_PATH, "strcXXXYX" + ".jpeg"), img.numpy())

        # plt.imshow(img)
        # plt.show()
       

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [9]:
base_model = applications.VGG19(
    include_top=False,
    input_shape=(*RECOG_IMAGE_SIZES, 3),
    weights='imagenet'
)
base_model.trainable = False


model = Sequential(base_model)
model.add(layers.Conv2D(256, (4, 4), activation='tanh', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(128, (4,4), activation='tanh', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(256, (4,4), activation='tanh', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(128, (4,4), activation='tanh', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(512, (4,4), activation='tanh', padding="same")) 
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(1024, (4,4), activation='tanh', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(512, (4,4), activation='tanh', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(1024, (4,4), activation='tanh', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='tanh'))
model.add(tf.keras.layers.Dense(1024, activation='tanh'))
model.add(tf.keras.layers.Dense(1024, activation='tanh'))
model.add(tf.keras.layers.Dense(LAST_DENSE_BEFORE_OUTPUT_NEURONAS, activation='tanh'))
model.add(layers.Dense(3, activation='softmax'))

In [10]:
model.compile(optimizer='adam', loss=losses.CategoricalCrossentropy(from_logits=False), metrics='accuracy', run_eagerly=True) 
model.fit(INIT_DATASET, epochs=6)

model.save(OUTPUT_FACES_RECOG)
tfjs.converters.save_keras_model(model, FACES_RECOG_MODEL_EXPORT)


Epoch 1/6
78/78 [==============================] - 16s 174ms/step - loss: 0.9783 - accuracy: 0.5096
Epoch 2/6
78/78 [==============================] - 14s 174ms/step - loss: 0.7156 - accuracy: 0.5545A: 0s - loss: 0.7159 - accuracy: 
Epoch 3/6
78/78 [==============================] - 14s 175ms/step - loss: 0.7137 - accuracy: 0.5096
Epoch 4/6
78/78 [==============================] - 14s 177ms/step - loss: 0.7058 - accuracy: 0.4712
Epoch 5/6
78/78 [==============================] - 14s 176ms/step - loss: 0.7025 - accuracy: 0.5288
Epoch 6/6
78/78 [==============================] - 14s 176ms/step - loss: 0.7037 - accuracy: 0.5224s - loss: 0.7043 -  -


In [11]:
with open(os.path.join(FACES_RECOG_LABEL_MAP)):
    extend_label_map(1, 'unknown')
    extend_label_map(2, 'init_img_1')
    extend_label_map(3, 'init_img_2')
